# Analyse d'adresse email

L'analyse d'une adresse email est assez complexe, car elle intègre de nombreux facteurs.
Voici une liste des principaux contrôles à réaliser : 

- Recherche de l'adresse dans les bases de CTI, afin d'identifier si cette derniere est connue pour avoir déjà envoyé du SPAM, ou si elle est apparue dans une fuite de données ;
- Extraction du TLD afin d'identifier si ce dernier est connu dans les bases de CTI ;


Si un doute persiste

Saisir l'adresse email à analyser : 

In [ ]:
suspicious_email = "XXXXxxxx@gmail.com"

# Initialisation

Cette section a pour objectif d'initialiser les modules pythons nécessaires pour ce notebook.

In [ ]:
# Python modules
from colorama import init, Fore, Back, Style
## EMAILRep module
# pip install emailrep
from emailrep import EmailRep

# Load config file
import sys
sys.path.append('..')
import config

## APIKEY
EMAILREP_APIKEY=config.EMAILREP_APIKEY

# Analyse

## EmailRep

Ce site permet d'analyser très rapidement une adresse mail. \
Il est possible de demander gratuitement une API KEY directement sur le site. \
reférence : https://emailrep.io

In [ ]:
# setup your api key (optional)
emailrep = EmailRep(EMAILREP_APIKEY)

# query an email address
report = emailrep.query(suspicious_email)

# Display report
for key in ['spam','blacklisted','credentials_leaked','data_breach','suspicious_tld','spoofable']:
    if (report['details'][key]):  
        print(f"{Fore.RED}[!] {key.upper()}") 
    else:
        print(f"{Fore.GREEN}[✓] Not {key.upper()}")

## Threat Intel 

L'adresse email comporte obligatoirement un nom de domaine. L'analyse de ce dernier au sein de bases de connaissances permettra de faciliter sa qualification.

Ici nous réaliserons une extraction du nom de domaine via le module msticpy et nous le rechercherons dans les bases de données de VirusTotal et Alienvault OTX.

In [ ]:
import pandas as pd

!export MSTICPYCONFIG=./msticpyconfig.yaml
from msticpy.transform.iocextract import IoCExtract
import msticpy as mp
import msticpy.sectools as sectools
mp.init_notebook(globals(), verbosity=0)
ti = mp.TILookup()
ti.loaded_providers
ioc_extractor = sectools.IoCExtract()

# Parse email and set dataframe
df_observable_str = ioc_extractor.extract(suspicious_email)
df_observable = pd.DataFrame.from_dict(df_observable_str, orient='index', columns=['Observable'])

# Research observables in theat intel database
df_observable_ti = ti.lookup_iocs(data=df_observable, ioc_col="Observable", providers=["VirusTotal", "OTX"])
df_observable_ti = pd.json_normalize(data=df_observable_ti[['Ioc','Provider','Details']].to_dict(orient='records'))

if (df_observable_ti.empty):
    print(Fore.GREEN + "[✓] No Potentially Suspicious Network connections identified")
else:
    print(Fore.RED + "[!] Potentially Suspicious Network connections identified")
    display(df_observable_ti.sort_values(by="Details.pulse_count", ascending=False))